# 4D tomography using the target imaging system

In [ ]:
import sys
import os
from os.path import join

import importlib
from tqdm import tqdm
from tqdm import trange
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import animation
import proplot as pplt

import target_image_analysis

sys.path.append('/Users/46h/Research/')
from accphys.tools import beam_analysis as ba
from accphys.tools import utils
from accphys.tools import plotting as myplt
from accphys.tools.accphys_utils import V_matrix_4x4_uncoupled
from accphys.emittance_measurement_4D.tomography import reconstruct as rec

In [ ]:
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['axes.grid'] = False
pplt.rc['grid.alpha'] = 0.04
pplt.rc['savefig.transparent'] = True
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'dusk_r'
pplt.rc['savefig.dpi'] = 'figure'
pplt.rc['animation.html'] = 'jshtml'
pplt.rc['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'
savefig_kws = dict(dpi=300)

## Optics

In [ ]:
folder = join(
    '/Users/46h/Research/code/openxal-scripts/emittance_measurement_4D/',
    '_saved/2021-10-21/setting2/injturns400/turn400/'
)

In [ ]:
filenames = utils.list_files(join(folder, 'target_scan/data/'))

### Phase advances 

In [ ]:
phase_adv = np.loadtxt(join(folder, 'target_scan/data/phase_adv.dat'))

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2.25))
g1 = ax.plot(phase_adv[:, 0], marker='.')
g2 = ax.plot(phase_adv[:, 1], marker='.')
ax.legend([g1, g2], labels=[r"$\mu_x$", r"$\mu_y$"], loc=(0., 1.), framealpha=0.)
ax.format(ylabel='[rad]', xlabel='Scan index', ylim=(0., 2*np.pi), ygrid=True)
plt.savefig('_output/figures/phase_adv.png', **savefig_kws)
plt.show()

### Twiss parameters vs. position

In [ ]:
twiss_filenames = [f for f in filenames if 'twiss' in f]
twiss_filenames = sorted(twiss_filenames, key=lambda f: float(f.split('.')[0].split('_')[-1]))
twiss_list = np.array([np.loadtxt(f) for f in twiss_filenames])

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2.25))
plot_kws = dict(alpha=0.15, marker=None, lw=None)
colors = myplt.DEFAULT_COLORCYCLE

for twiss in twiss_list:
    positions, mux, muy, alpha_x, alpha_y, beta_x, beta_y, eps_x, eps_y = twiss.T
    ax.plot(positions, beta_x, color=colors[0], **plot_kws)
    ax.plot(positions, beta_y, color=colors[1], **plot_kws)
ax.format(ylabel='[m/rad]', ylim=(0., 100.), xlabel='Position [m]', ygrid=True)

lines = [Line2D([0], [0], color=colors[0]), Line2D([0], [0], color=colors[1])]
ax.legend(lines, [r"$\beta_x$", r"$\beta_y$"], loc=(0., 1.), framealpha=0.)
plt.savefig('_output/figures/beta_vs_pos.png', **savefig_kws)
plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2.25))
for twiss in twiss_list:
    positions, mu_x, mu_y, alpha_x, alpha_y, beta_x, beta_y, eps_x, eps_y = twiss.T
    ax.plot(positions, mu_x, color=colors[0], **plot_kws)
    ax.plot(positions, mu_y, color=colors[1], **plot_kws)
ax.format(ylabel='[m/rad]', xlabel='Position [m]', ylim=(0., 2*np.pi), ygrid=True)

lines = [Line2D([0], [0], color=colors[0]), Line2D([0], [0], color=colors[1])]
ax.legend(lines, [r"$\mu_x$", r"$\mu_y$"], loc=(0., 1.), framealpha=0.)
plt.savefig('_output/figures/phase_adv_vs_pos.png', **savefig_kws)
plt.show()

### Quadrupole field strengths 

In [ ]:
fields = pd.read_table(join(folder, 'target_scan/data/fields.dat'), sep=' ')

In [ ]:
fig, ax = pplt.subplots()
ax.format(ylabel='Field strength [T/m]', cycle='538', xlabel='Scan index', ygrid=True)
for col in fields.columns[:-1]:
    ax.plot(fields.loc[:, col], marker='.', ms=3)
ax.legend(ncols=1, loc='r')
plt.savefig('_output/figures/fields.png', **savefig_kws)
plt.show()

### Comparison with wire-scanner measurements

Load the model Twiss parameters at each node.

In [ ]:
model_twiss = pd.read_table(join(folder, 'model_twiss.dat'), sep=' ')
model_twiss

Load the reconstructed beam moments at each node. These were reconstructed from wire-scanner measurements. We should ignore everything downstream of `RTBT_Diag:QH18` because the magnets downstream were changed during the target scan.

In [ ]:
rec_moments = pd.read_table(join(folder, 'rec_moments.dat'), sep=' ')
rec_moments

In [ ]:
rec_stats = ba.StatsReader()
rec_stats.read_moments(rec_moments.iloc[:, 2:].values)
rec_stats.twiss2D

In [ ]:
ws_ids = ['RTBT_Diag:WS20', 'RTBT_Diag:WS21', 'RTBT_Diag:WS23', 'RTBT_Diag:WS24']
ws_positions = []
for ws_id in ws_ids:
    ws_position = float(rec_moments[rec_moments['node_id'] == ws_id]['position']) 
    ws_positions.append(ws_position)

In [ ]:
fig, axes = pplt.subplots(nrows=4, figsize=(3, 5), spany=False)
colors = pplt.Cycle('538').by_key()['color']
plot_kws = dict(marker='.', ms=0)
axes[0].plot(rec_moments['position'], rec_stats.corr['xy'], color=colors[0], **plot_kws)
axes[1].plot(rec_moments['position'], rec_stats.corr['xyp'], color=colors[1], **plot_kws)
axes[2].plot(rec_moments['position'], rec_stats.corr['yxp'], color=colors[2], **plot_kws)
axes[3].plot(rec_moments['position'], rec_stats.corr['xpyp'], color=colors[3], **plot_kws)
axes.format(xlabel='position', ylim=(-1.0, 1.0), suptitle='Cross-plane correlation coefficients.')
axes[0].set_ylabel("x-y")
axes[1].set_ylabel("x-y'")
axes[2].set_ylabel("y-x'")
axes[3].set_ylabel("x'-y'")
for ax in axes:
    for ws_position in ws_positions:
        ax.axvline(ws_position, color='black', alpha=0.2, ls='--', lw=0.5)
plt.savefig('_output/figures/corr_coeff.png', **savefig_kws)
plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(6.5, 2.5))
rec_kws = dict(ls='--', lw=0.75)
model_kws = dict(marker='.', ms=3, lw=1.25)
colors = myplt.DEFAULT_COLORCYCLE
g1 = ax.plot(model_twiss['position'], model_twiss['beta_x'], color=colors[0], **model_kws)
g2 = ax.plot(rec_twiss['position'], rec_twiss['beta_x'], color=colors[0], **rec_kws)
g3 = ax.plot(model_twiss['position'], model_twiss['beta_y'], color=colors[1], **model_kws)
g4 = ax.plot(rec_twiss['position'], rec_twiss['beta_y'], color=colors[1], **rec_kws)
ax.legend(handles=[g1, g2, g3, g4], 
          labels=[r'$\beta_x$ (model)', r'$\beta_x$ (rec)',
                  r'$\beta_y$ (model)', r'$\beta_y$ (rec)'], 
          ncols=2, loc='upper left')
ax.format(xlabel='Position [m]', ylabel='[m/rad]')
plt.savefig('_output/figures/rec_betas.png', **savefig_kws)
plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(6.5, 2.5))
rec_kws = dict(ls='--', lw=0.75)
colors = myplt.DEFAULT_COLORCYCLE
g1 = ax.plot(model_twiss['position'], model_twiss['alpha_x'], color=colors[0], **model_kws)
g2 = ax.plot(rec_moments['position'], rec_stats.twiss2D['alpha_x'], color=colors[0], **rec_kws)
g3 = ax.plot(model_twiss['position'], model_twiss['alpha_y'], color=colors[1], **model_kws)
g4 = ax.plot(rec_moments['position'], rec_stats.twiss2D['alpha_y'], color=colors[1], **rec_kws)
ax.legend(handles=[g1, g2, g3, g4], 
          labels=[r'$\alpha_x$ (model)', r'$\alpha_x$ (rec)',
                  r'$\alpha_y$ (model)', r'$\alpha_y$ (rec)'], 
          ncols=2, loc='upper left')
ax.format(xlabel='Position [m]', ylabel='[rad]')
plt.savefig('_output/figures/rec_alphas.png', **savefig_kws)
plt.show()

The measured Twiss parameters seem to agree reasonably well with the model at `RTBT_Diag:BPM16`. We will use this as our reconstruction location; although the method functions the same for any reconstruction location, choosing a point where the beam is matched will make the projection angles close to the phase advances in normalized phase space. Also, the reconstructed distribution will be more round in normalized phase space.

In [ ]:
rec_node_id = 'RTBT_Diag:BPM16'

idx = model_twiss['node_id'] == rec_node_id
rec_node_id, position, alpha_x, alpha_y, beta_x, beta_y = model_twiss.loc[idx].values[0]
print('rec_node_id = {} [m]'.format(rec_node_id))
print('position = {:.2f}'.format(position))
print('alpha_x = {:.2f} [rad]'.format(alpha_x))
print('alpha_y = {:.2f} [rad]'.format(alpha_y))
print('beta_x = {:.2f} [m/rad]'.format(beta_x))
print('beta_y = {:.2f} [m/rad]'.format(beta_y))

V = V_matrix_4x4_uncoupled(alpha_x, alpha_y, beta_x, beta_y)
Vx = V[:2, :2]
Vy = V[2:, 2:]
print('V =')
print(V)

### Transfer matrices 

Load the transfer matrices from this node to the target for each optics setting.

In [ ]:
tmat_filenames = [f for f in filenames if 'tmat' in f]
tmat_filenames = sorted(tmat_filenames, key=lambda f: float(f.split('.')[0].split('_')[-1]))
tmats = []
for filename in tmat_filenames:
    file = open(filename, 'r')
    lines = [line.rstrip() for line in file]
    for line in lines[1:]:
        tokens = line.rstrip().split()
        node_id, tmat_elems = tokens[0], [float(token) for token in tokens[1:]]
        if node_id == rec_node_id:
            tmats.append(np.array(tmat_elems).reshape((4, 4)))
    file.close()

## Target images 

In [ ]:
image_filenames = [f for f in filenames if 'image' in f]
image_filenames = list(sorted(image_filenames, key=lambda f: int(f.split('.dat')[0].split('_')[-1])))
images = []
for image_filenames in tqdm(image_filenames):
    image = target_image_analysis.read_file(image_filenames, n_avg='all', make_square=True, thresh=200)
    images.append(image)

Plot the images.

In [ ]:
# fig, ax = pplt.subplots(figsize=(4.2, 2.5))
# ax.format(xlabel='x [mm]', ylabel='y [mm]', aspect=1.0)
    
# for ellipse in [True, False]:
#     for sigma in [0.0, 4.0]:
#         for i, image in enumerate(tqdm(images)):
#             fstr = r'Step {}: $\mu_x = {:.2f}\degree$, $\mu_y = {:.2f}\degree$.'
#             ax.set_title(fstr.format(i, np.degrees(phase_adv[i, 0]), np.degrees(phase_adv[i, 1])))

#             image.filter(sigma)
#             q = ax.pcolormesh(image.xx, image.yy, image.Zf.T, cmap='mono_r')

#             for patch in ax.patches:
#                 patch.set_visible(False)
#             if ellipse:
#                 mean_x, mean_y, sig_xx, sig_yy, sig_xy = image.estimate_moments(use_filtered=sigma==0.0)
#                 c1, c2, angle = target_image_analysis.rms_ellipse_dims(sig_xx, sig_yy, sig_xy)  
#                 myplt.ellipse(ax, 2 * c1, 2 * c2, angle, center=(mean_x, mean_y), 
#                               color='red8', lw=None, alpha=0.25)
                
#                 r = sig_xy / np.sqrt(sig_xx * sig_yy)
#                 for text in ax.texts:
#                     text.set_visible(False)
#                 ax.annotate('r = {:.2f}'.format(r), xy=(0.02, 0.92), xycoords='axes fraction', color='white',
#                             fontsize='small')

#             plt.savefig('_output/figures/target_image_sigma={}_ellipse={}_{}.png'.format(sigma, ellipse, i), **savefig_kws)
#             q.set_visible(False)

The $x$-$y$ correlation coefficient can be computed directly from the image.

In [ ]:
sigma = 4.0
cor_coefs = []
for image in images:
    image.filter(sigma)
    mean_x, mean_y, sig_xx, sig_yy, sig_xy = image.estimate_moments(use_filtered=sigma==0.0)
    r = sig_xy / np.sqrt(sig_xx * sig_yy)
    cor_coefs.append(r)

In [ ]:
fig, axes = pplt.subplots(nrows=2, figsize=(3.5, 3.5), height_ratios=[1.0, 1.0], spany=False)
axes[0].plot(cor_coefs, marker='.', color='black')
ymax = max(np.abs(axes[0].get_ylim()))
axes[0].format(ylim=(-ymax, ymax), ylabel='x-y cor. coef.')
g1 = axes[1].plot(phase_adv[:, 0], marker='.')
g2 = axes[1].plot(phase_adv[:, 1], marker='.')
axes[1].legend([g1, g2], labels=[r"$\mu_x$", r"$\mu_y$"], loc='upper left')
axes[1].format(ylabel='[rad]', ylim=(0.0, 2.0 * np.pi))
axes.format(xlabel='Scan index', ygrid=True, xgrid=False)
plt.savefig('_output/figures/cor_coef.png', **savefig_kws)
plt.show()

## 4D reconstruction from multiple 2D reconstructions

### Form measurement array 

The transfer matrix $M$ connects the phase space coordinates at the reconstruction location (1) and measurement location (2):

$$ u_2 = M u_1. $$

We can perform the reconstruction in normalized phase space. We have 

$$\begin{aligned}
    u_2 &= M u_1, \\
    V_{2}^{-1} u_2 &= V_{2}^{-1} M V_{1} V_{1}^{-1} u_1, \\
    u_2 &= M V_{1} u_{1n}, \\
\end{aligned}$$

where $V_i = V_i(\alpha_i, \beta_i)$ is the normalization matrix at position $i$. Defining $\tilde{M} = M V_{1}$ gives

$$ u_2 = \tilde{M} u_{1n}.$$

We then proceed with the reconstruction method as normal with $\tilde{M}$ so that the projections are converted to normalized phase space at position 1.  

Collect the transfer matrices for each of these optics settings. There should be 7 matrices for each plane.

In [ ]:
n_steps_per_dim = 6

In [ ]:
x_idx = [i * n_steps_per_dim for i in range(n_steps_per_dim)]
y_idx = [i for i in range(n_steps_per_dim)]

We have to do a little extra work because the optics calculation was done incorrectly in the last study. The corrections were done on top of additional corrections from the wire-scanner measurement, so a couple settings failed to maintain a constant $\mu_x$ while $\mu_y$ was changed. It was really just two steps (see plot of phase advances during scan). Vertical optics were still perfect. We will neglect this error since it is very small. 

In [ ]:
x_idx[5] += 2

In [ ]:
tmats_x = [tmats[i][:2, :2] for i in x_idx]
tmats_y = [tmats[i][2:, 2:] for i in y_idx]

Create the measurement array $S$ such that $S_{i, j, k, l}$ gives the intensity at position ($x = x_i$, $y = y_j$) on the screen for the $k$th set of $x$ optics and $l$th set of $y$ optics. We'll need to downsize the image resolution.

In [ ]:
from skimage import transform

In [ ]:
n_bins = 100
sigma = 4.0

S = np.zeros((n_bins, n_bins, len(tmats_x), len(tmats_y)))
scan_index = 0
for k in range(len(tmats_x)):
    for l in range(len(tmats_y)):
        if scan_index > len(images) - 1:
            break    
        image = images[scan_index]
        Z = image.filter(sigma)        
        Z = transform.resize(Z, (n_bins, n_bins), anti_aliasing=True)

        
        xmax = max(image.xx)
        grid = 2 * [np.linspace(-xmax, xmax, n_bins)]
        xmax_new = 0.6 * xmax
        new_grid = 2 * [np.linspace(-xmax_new, xmax_new, n_bins)]
        Z, _ = rec.transform(Z, np.identity(2), grid, new_grid)
        
        S[:, :, k, l] = Z
        scan_index += 1

### Test projection scaling 

Given the projection at location 2, we can find the corresponding projection angle $\theta$ at location 1:

$$ \tan\theta = M_{11} / M_{12}.$$

We can also find the scaling between $s$, the distance along the projection axis at location 1 in normalized phase space, and $x_2$:

$$ a \equiv \frac{x_2}{s} = \sqrt{M_{11}^2 + M_{12}^2} .$$

Let $p$ be the projection. Then

$$ p_1(s) = a p_2(x_2) .$$

In [ ]:
def transfer_matrix_scale_factor(M):
    return np.sqrt(M[0, 0]**2 + M[0, 1]**2)

def transfer_matrix_angle(M):
    return np.arctan(M[0, 1] / M[0, 0])

First test on rotation matrices.

In [ ]:
theta_xs, theta_ys = [], []
for mux, muy in zip(phase_adv[x_idx, 0], phase_adv[y_idx, 1]):
    Nx = utils.rotation_matrix(mux)
    Ny = utils.rotation_matrix(muy)    
    theta_xs.append(transfer_matrix_angle(Nx))
    theta_ys.append(transfer_matrix_angle(Ny)) 
    
theta_xs = np.array(theta_xs)
theta_ys = np.array(theta_ys)
theta_xs[np.where(theta_xs < 0.)] += np.pi
theta_ys[np.where(theta_ys < 0.)] += np.pi
    
fig, ax = pplt.subplots()
ax.format(yformatter='deg', xtickminor=False)
ax.plot(np.degrees(theta_xs), label=r'$\theta_x$')
ax.plot(np.degrees(theta_ys), label=r'$\theta_y$')
ax.plot(np.degrees(phase_adv[x_idx, 0] % np.pi), lw=0, marker='+', ms=10, color='black')
ax.plot(np.degrees(phase_adv[y_idx, 1] % np.pi), lw=0, marker='+', ms=10, color='black')
ax.legend(ncols=2, loc='top')
plt.show()

Now calculate the real angles.

In [ ]:
theta_xs, theta_ys = [], []
for Mx, My in zip(tmats_x, tmats_y):
    Nx = np.matmul(Mx, Vx)
    Ny = np.matmul(My, Vy)
    theta_xs.append(transfer_matrix_angle(Nx))
    theta_ys.append(transfer_matrix_angle(Ny))   
        
theta_xs = np.array(theta_xs)
theta_ys = np.array(theta_ys)
theta_xs[np.where(theta_xs < 0.)] += np.pi
theta_ys[np.where(theta_ys < 0.)] += np.pi
    
fig, ax = pplt.subplots()
ax.format(yformatter='deg', xtickminor=False)
ax.plot(np.degrees(theta_xs), marker='.', label=r'$\theta_x$')
ax.plot(np.degrees(theta_ys), marker='.', label=r'$\theta_y$')
ax.legend(ncols=2, loc='top')
plt.show()

### Reconstruct x-x'-y

In [ ]:
K = len(tmats_x)
L = len(tmats_y)

D = np.zeros((n_bins, L, n_bins, n_bins))
for j in trange(n_bins):
    for l in range(L):
        projections = S[:, j, :, l]
        
        # Get projections in normalized phase space at reconstruction location.
        angles = []
        for k in range(K):
            Mx = np.matmul(tmats_x[k], Vx)
            projections[:, k] *= transfer_matrix_scale_factor(Mx)
            theta = transfer_matrix_angle(Mx)
            if theta < 0.:
                theta += np.pi
            angles.append(theta)    
            
        # Reconstruct x-x'.
        D[j, l, :, :] = rec.sart(projections, angles, iterations=2)

### Reconstruct x-x'-y-y'

In [ ]:
Z = np.zeros((n_bins, n_bins, n_bins, n_bins))

for r in trange(n_bins):
    for s in range(n_bins):
        projections = D[:, :, r, s]
        
        angles = []
        for l in range(L):
            My = np.matmul(tmats_y[l], Vy)
            projections[:, l] *= transfer_matrix_scale_factor(My)
            theta = transfer_matrix_angle(My)
            if theta < 0.:
                theta += np.pi
            angles.append(theta)    
        
        # Reconstruct y-y'.
        Z[r, s, :, :] = rec.sart(projections, angles, iterations=2)

In [ ]:
Z = np.clip(Z, 0.0, None)

In [ ]:
proj_2D = [[], [], []]
for i in range(3):
    for j in range(i + 1):
        proj_2D[i].append(rec.project(Z, [j, i + 1]))

        
joint_kws = dict(cmap='mono_r')
line_kws = dict(lw=1)
labels = [r"$x_n$", r"$x'_n$", r"$y_n$", r"$y'_n$"]

fig, axes = myplt.pair_grid_nodiag(4, figsize=(7, 7))
for i in range(3):
    for j in range(i + 1):
        ax = axes[i, j]
        _Z = proj_2D[i][j]
#         xlim, ylim = ax.get_xlim(), ax.get_ylim()
#         xx = np.linspace(xlim[0], xlim[1], Z.shape[0])
#         yy = np.linspace(ylim[0], ylim[1], Z.shape[1])
#         ax.pcolormesh(xx, yy, Z.T, **joint_kws)
        ax.pcolormesh(_Z.T, **joint_kws)
    
#         ax.axvline(50, color='red', **line_kws)
#         ax.axhline(50, color='red', **line_kws)
    
# for ax in axes.flat:
#     ax.set_xlim(20, 80)
#     ax.set_ylim(20, 80)

* What are the dimensions of the reconstruction grid? We need this to transform to the actual phase space coordinates.
* I think we need more projections to get rid of streaking.
* We may need to center the measured projections; some appear to be off-center here. 

## Direct 4D reconstruction 

### ART 

### MENT 

### PIC iteration